## what are we trying to do
a graph with 2 agents. one that writes an article plan from a raw interview with an expert 
another agent that writes the blogpost from the plan 
## next steps (later)
try to add interrupt
and human feedback 


In [1]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o", temperature=0) 

In [3]:
_set_env("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "meetup_writing_agent"

In [4]:
# Cell 7: Updated Company Strategy Context Loader (3 Documents)
def load_company_strategy_context():
    """Load company strategy, SEO strategy, and content strategy for context"""
    
    strategy_context = {}
    
    try:
        # Load company strategy
        company_strategy_path = "../data/processed/company_strategy.mkd"
        if os.path.exists(company_strategy_path):
            with open(company_strategy_path, "r", encoding="utf-8") as f:
                strategy_context["company_strategy"] = f.read()
            print(f"✅ Loaded company strategy ({len(strategy_context['company_strategy'])} chars)")
        else:
            strategy_context["company_strategy"] = "Company strategy document not available."
            print("⚠️ Company strategy document not found")
        
        # Load SEO strategy
        seo_strategy_path = "../data/processed/seo_strategy.mkd"
        if os.path.exists(seo_strategy_path):
            with open(seo_strategy_path, "r", encoding="utf-8") as f:
                strategy_context["seo_strategy"] = f.read()
            print(f"✅ Loaded SEO strategy ({len(strategy_context['seo_strategy'])} chars)")
        else:
            strategy_context["seo_strategy"] = "SEO strategy document not available."
            print("⚠️ SEO strategy document not found")
        
        # Load content strategy (NEW)
        content_strategy_path = "../data/processed/content_strategy.mkd"
        if os.path.exists(content_strategy_path):
            with open(content_strategy_path, "r", encoding="utf-8") as f:
                strategy_context["content_strategy"] = f.read()
            print(f"✅ Loaded content strategy ({len(strategy_context['content_strategy'])} chars)")
        else:
            strategy_context["content_strategy"] = "Content strategy document not available."
            print("⚠️ Content strategy document not found")
            
    except Exception as e:
        print(f"❌ Error loading strategy documents: {e}")
        strategy_context = {
            "company_strategy": "Strategy document not available",
            "seo_strategy": "SEO strategy document not available", 
            "content_strategy": "Content strategy document not available"
        }
    
    return strategy_context

# Test loading all three documents
strategy_context = load_company_strategy_context()
print(f"📊 Strategy context keys: {list(strategy_context.keys())}")
print(f"📊 Total context size: {sum(len(v) for v in strategy_context.values() if isinstance(v, str))} chars")

✅ Loaded company strategy (6555 chars)
✅ Loaded SEO strategy (1120 chars)
✅ Loaded content strategy (4469 chars)
📊 Strategy context keys: ['company_strategy', 'seo_strategy', 'content_strategy']
📊 Total context size: 12144 chars


In [5]:
from typing import List
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

class Plan(BaseModel):
    who: str = Field(
        description="target reader of the blog post, e.g., 'AI researchers', 'business leaders', etc.",
    )
    why: str = Field(
        description="why are we writing this blog post.",
    )
    what: str = Field(
        description="what are the main topics to cover in the blog post.",
    )
    the_issue: str = Field(
        description="the main issue or problem that the blog post addresses.",
    )
    where_we_stand: str = Field(
        description="current position or perspective on the issue.",
    )
    single_message: str = Field(
        description="the single most important message to convey in the blog post.",
    )

    @property
    def plan(self) -> str:
        return (
            f"Who: {self.who}\n"
            f"Why: {self.why}\n"
            f"What: {self.what}\n"
            f"Issue: {self.the_issue}\n"
            f"Where We Stand: {self.where_we_stand}\n"
            f"Single Message: {self.single_message}\n"
        )

class Perspectives(BaseModel):
    plan: List[Plan] = Field(
        description="AI generated plan / scaffold for the blog post.",
    )

class BlogPost(BaseModel):
    title: str = Field(description="The title of the blog post.")
    content: str = Field(description="The full content of the blog post.")

    
class GenerateBlogState(TypedDict):
    company_strategy: str  # Company strategy
    content_strategy: str      # content strategy
    plan: List[Plan]       # AI generated plan / scaffold for the blog post.
    human_analyst_feedback: str # Human feedback
    blog_post: List[BlogPost]  # the substance of the blog posts, with a {"title": str, "content": str} written according to the plan.

In [ ]:
from IPython.display import Image, display
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

# Step 1: Load the strategy context using load_company_strategy_context()
strategy_context = load_company_strategy_context()

# Step 2: Define the instruction template with placeholders
plan_instructions = """You are tasked with creating a plan for a professional blog post for a company called Big Kids Automation Agency. The plan is just a skeleton of a blog post which contains
some questions and answers that will help the company writer to write a focused blog post. This plan is a set of instructions to the writer to stay focused and write the post.


Follow these instructions carefully:

1. First, review the company business strategy. Examine the company strategy:
{company_strategy_content}

2. The blog post must fit the company content strategy. Examine the company content strategy:
{content_strategy_content}

3. The blog post must fit the company SEO strategy. Examine the company SEO strategy:
{seo_strategy_content}

4. Create a plan for a blog post. The plan should include the following sections:
who, why, what, the_issue, where_we_stand, single_message.

5. Examine any editorial feedback from the human analyst. If there is any feedback, incorporate it into the plan:
{human_analyst_feedback}
"""

# Step 3: Format the instructions with the loaded content and other variables
# If they aren't defined yet, you can pass empty strings or placeholders for now.
formatted_instructions = plan_instructions.format(
    company_strategy_content=strategy_context.get('company_strategy', 'Company strategy not available'),
    content_strategy_content=strategy_context.get('content_strategy', 'Content strategy not available'),
    seo_strategy_content=strategy_context.get('seo_strategy', 'SEO strategy not available'),
    raw_input=raw_input if 'raw_input' in globals() else "Raw input not available",  # Replace with actual variable if defined
    human_analyst_feedback=human_analyst_feedback if 'human_analyst_feedback' in globals() else "No feedback provided"  # Replace with actual variable if defined
)

# Step 4: Optionally log to confirm content loading
print("✅ Strategy context loaded for creative agent:")
print(f"   Company strategy: {len(strategy_context.get('company_strategy', ''))} chars")
print(f"   Content strategy: {len(strategy_context.get('content_strategy', ''))} chars")
print(f"   SEO strategy: {len(strategy_context.get('seo_strategy', ''))} chars")

# Step 5: Use formatted_instructions as needed (e.g., pass to a creative agent or LLM)
# Example: agent.process(formatted_instructions)

✅ Loaded company strategy (6555 chars)
✅ Loaded SEO strategy (1120 chars)
✅ Loaded content strategy (4469 chars)
✅ Strategy context loaded for creative agent:
   Company strategy: 6555 chars
   Content strategy: 4469 chars
   SEO strategy: 1120 chars


In [23]:
def create_plan(state: GenerateBlogState):
    """Create plan"""
    company_strategy = state['company_strategy']
    content_strategy = state['content_strategy']
    human_analyst_feedback=state.get('human_analyst_feedback', '')

    
    # Read the company strategy
    try:
        with open("company_strategy_path", "r", encoding="utf-8") as f:
            company_strategy = f.read()
    except FileNotFoundError:
        raw_input = "No raw input file found."
    
    # Read the content strategy
    try:
        with open("content_strategy_path", "r", encoding="utf-8") as f:
            content_strategy = f.read()
    except FileNotFoundError:
        raw_input = "No raw input file found."
    
    # Read raw_input file
    try:
        with open("raw_input_path", "r", encoding="utf-8") as f:
            raw_input = f.read()
    except FileNotFoundError:
        raw_input = "No raw input file found."

    

    # Enforce structured output
    structured_llm = llm.with_structured_output(Perspectives)

    # System message
    system_message = plan_instructions.format(
        company_strategy=company_strategy,
        human_analyst_feedback=human_analyst_feedback,
        content_strategy=content_strategy,
        raw_input=raw_input
    )

    # Generate question 
    plan = structured_llm.invoke([SystemMessage(content=system_message)] + [HumanMessage(content="Generate a possible article plan to help the company create a blogpost that resonates in the target reader.")])
    
    # Write the plan to the state
    return {"plan": plan.plan}

def human_analyst_feedback(state: GenerateBlogState):
    """ No-op node that should be interrupted on """
    pass

def should_continue(state: GenerateBlogState):
    """ Return the next node to execute """

    # Check if human feedback
    human_analyst_feedback=state.get('human_analyst_feedback', None)
    if human_analyst_feedback:
        return "create_plan"
    
    # Otherwise go to the next node
    return "write_blog_post"
   
def write_blog_post(state: GenerateBlogState):
    """Generate a blog post draft from the plan."""
    
    blog_post_instructions = (
        "You are a professional blog writer for Big Kids Automation Agency. "
        "Using the following plan, write a complete blog post. "
        f'Plan: {state["plan"]}\n'

        "Make sure the post is engaging, clear, and follows the structure and intent of the plan. "
        "Return a title and the full content of the blog post.\n\n"
    )

    structured_llm = llm.with_structured_output(BlogPost)
    blog_post = structured_llm.invoke([SystemMessage(content=blog_post_instructions), HumanMessage(content="Write the blog post.")])
    # Return the blog_post as part of the state
    return {"blog_post": [blog_post]}

In [8]:
# Add nodes and edges 
builder = StateGraph(GenerateBlogState)
builder.add_node("create_plan", create_plan)
builder.add_node("human_analyst_feedback", human_analyst_feedback)
builder.add_node("write_blog_post", write_blog_post)

builder.add_edge(START, "create_plan")
builder.add_edge("create_plan", "human_analyst_feedback")
builder.add_conditional_edges("human_analyst_feedback", should_continue, ["create_plan", "write_blog_post"])
builder.add_edge("write_blog_post", END)

# Compile
memory = MemorySaver()
graph = builder.compile(interrupt_before=['human_analyst_feedback'], checkpointer=memory)




In [21]:
# Input
thread = {"configurable": {"thread_id": "4"}}

# Run the graph until the first interruption



try:
    for event in graph.stream({"company_strategy": company_strategy_path, "content_strategy": content_strategy_path}, thread, stream_mode="values"):
        # Review
        plan = event.get('plan', '')
        blog_post = event.get('blog_post', None)
            
        if plan:
            print("=== PLAN GENERATED ===")
            for p in plan:
                print(f"Who: {p.who}")
                print(f"Why: {p.why}")
                print(f"What: {p.what}")
                print(f"Issue: {p.the_issue}")
                print(f"Where We Stand: {p.where_we_stand}")
                print(f"Single Message: {p.single_message}")
                print("-" * 50)
        
        if blog_post:
            print("=== BLOG POST GENERATED ===")
            if isinstance(blog_post, list) and blog_post:
                post = blog_post[0]
                print("Title:", post.title)
                print("\nContent:\n", post.content)
            elif hasattr(blog_post, 'title'):
                print("Title:", blog_post.title)
                print("\nContent:\n", blog_post.content)
            print("=== GENERATION COMPLETE ===")
except Exception as e:
    print(f"Error during graph execution: {e}")
    # Check the current state
    state = graph.get_state(thread)
    print(f"Current state keys: {state.values.keys() if state.values else 'No state'}")
    print(f"Next node: {state.next}")
    print(f"Error during graph execution: {e}")
    # Check the current state
    state = graph.get_state(thread)
    print(f"Current state keys: {state.values.keys() if state.values else 'No state'}")
    print(f"Next node: {state.next}")

=== PLAN GENERATED ===
Who: Business leaders and decision-makers in the automation industry
Why: To inform and educate our audience about the latest trends and strategies in automation, aligning with our mission to be thought leaders in the industry.
What: The blog post should cover the importance of automation in modern business, the latest trends in automation technology, and how businesses can effectively implement these technologies to improve efficiency and productivity.
Issue: Many businesses are struggling to keep up with the rapid advancements in automation technology and are unsure how to integrate these solutions effectively into their operations.
Where We Stand: Big Kids Automation Agency believes in a strategic and tailored approach to automation, ensuring that each solution is customized to meet the specific needs of the business.
Single Message: Automation is not just a trend; it's a necessity for businesses looking to stay competitive in today's fast-paced market.
------

In [26]:
# Get state and look at next node
state = graph.get_state(thread)
state.next

()

In [19]:

graph.update_state(thread, {"human_analyst_feedback": 
                            "please include a reference to security issues"}, as_node="human_analyst_feedback")

{'configurable': {'thread_id': '4',
  'checkpoint_ns': '',
  'checkpoint_id': '1f08c8ae-3927-6031-800b-7c91e6407466'}}

In [27]:
# Provide "no feedback" or completion signal
graph.update_state(thread, {"human_analyst_feedback": None}, 
                  as_node="human_analyst_feedback")

{'configurable': {'thread_id': '4',
  'checkpoint_ns': '',
  'checkpoint_id': '1f08c8fa-5bd2-6b62-8011-14167cc918b7'}}

In [25]:
# Resume execution
result = graph.invoke(None, config=thread)

In [28]:
try:
    for event in graph.stream({"company_strategy": company_strategy_path, "content_strategy": content_strategy_path}, thread, stream_mode="values"):
        # Review
        plan = event.get('plan', '')
        blog_post = event.get('blog_post', None)
            
        if plan:
            print("=== PLAN GENERATED ===")
            for p in plan:
                print(f"Who: {p.who}")
                print(f"Why: {p.why}")
                print(f"What: {p.what}")
                print(f"Issue: {p.the_issue}")
                print(f"Where We Stand: {p.where_we_stand}")
                print(f"Single Message: {p.single_message}")
                print("-" * 50)
        
        if blog_post:
            print("=== BLOG POST GENERATED ===")
            if isinstance(blog_post, list) and blog_post:
                post = blog_post[0]
                print("Title:", post.title)
                print("\nContent:\n", post.content)
            elif hasattr(blog_post, 'title'):
                print("Title:", blog_post.title)
                print("\nContent:\n", blog_post.content)
            print("=== GENERATION COMPLETE ===")
except Exception as e:
    print(f"Error during graph execution: {e}")
    # Check the current state
    state = graph.get_state(thread)
    print(f"Current state keys: {state.values.keys() if state.values else 'No state'}")
    print(f"Next node: {state.next}")
    print(f"Error during graph execution: {e}")
    # Check the current state
    state = graph.get_state(thread)
    print(f"Current state keys: {state.values.keys() if state.values else 'No state'}")
    print(f"Next node: {state.next}")

=== PLAN GENERATED ===
Who: Business leaders and decision-makers in the automation industry
Why: To inform and educate our audience about the latest trends and challenges in automation, and how Big Kids Automation Agency can provide solutions.
What: 1. Overview of current trends in automation.
2. Challenges faced by businesses in implementing automation.
3. The role of security in automation.
4. How Big Kids Automation Agency addresses these challenges.
5. Case studies or examples of successful automation projects.
Issue: Many businesses struggle with implementing automation effectively due to security concerns and lack of expertise.
Where We Stand: Big Kids Automation Agency is at the forefront of providing secure and efficient automation solutions tailored to business needs.
Single Message: Big Kids Automation Agency is your trusted partner in navigating the complexities of automation with a focus on security and efficiency.
--------------------------------------------------
=== BLOG

In [29]:
try:
    # Get final state directly
    final_state = graph.get_state(thread)
    blog_post = final_state.values.get('blog_post', None)
    
    if blog_post:
        if isinstance(blog_post, list) and blog_post:
            post = blog_post[0]
            print(f"Title: {post.title}")
            print(f"\nContent:\n{post.content}")
        elif hasattr(blog_post, 'title'):
            print(f"Title: {blog_post.title}")
            print(f"\nContent:\n{blog_post.content}")
    else:
        print("No blog post found")
        
except Exception as e:
    print(f"Error: {e}")

Title: Navigating the Future of Automation: Trends, Challenges, and Solutions

Content:
In the rapidly evolving world of automation, staying ahead of the curve is crucial for business leaders and decision-makers. As technology continues to advance, the automation industry is experiencing significant shifts that are reshaping how businesses operate. At Big Kids Automation Agency, we understand the complexities involved in implementing automation solutions and are committed to helping you navigate these changes with confidence and security.

### Current Trends in Automation

The automation landscape is being transformed by several key trends. Artificial Intelligence (AI) and Machine Learning (ML) are at the forefront, enabling more sophisticated and adaptive automation systems. These technologies allow for predictive maintenance, enhanced data analytics, and improved decision-making processes. Additionally, the rise of the Internet of Things (IoT) is creating interconnected systems that 